In [1]:
import graphlab as gl
import pandas as pd
import cPickle as pickle
import matplotlib.pyplot as plt
%matplotlib inline

This non-commercial license of GraphLab Create for academic use is assigned to mpmakris@gmail.com and will expire on June 30, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1470536484.log


In [2]:
gl.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 40)

### IMPORT TARGET INFO FROM PREPPER

In [ ]:
def open_prepper(file_path):
    """Open the DataPrepper from pickled file."""
    with open(file_path) as f:
        prepper = pickle.load(f)
    return prepper

file_path = '../data/store/data_prepper_BUILDING.pkl'
prepper = open_prepper(file_path)

X_train, y_train = prepper.return_training_data()
X_test, y_test = prepper.return_testing_data()

In [ ]:
y_combined = pd.concat((y_train, y_test), axis=0)

In [ ]:
plt.hist(y_combined['image_views_quantized'], bins=40)

### GRAPH LAB IMPORT IMAGES

In [ ]:
images_raw = gl.image_analysis.load_images('/home/ubuntu/data/images/BUILDING/')

In [ ]:
images_raw.head(1)

In [ ]:
images = images_raw

In [ ]:
images['filename'] = images['path'].apply(lambda x: x[x.find('/')+1:])

In [ ]:
check = True
while check:
    images['filename'] = images['filename'].apply(lambda x: x[x.find('/')+1:])
    if images[0]['filename'].find('/') < 0:
        check = False

In [ ]:
images['owner'] = images['filename'].apply(lambda x: x[:x.find('_')])
images['id'] = images['filename'].apply(lambda x: x[x.find('_')+1:])
images['id'] = images['id'].apply(lambda x: x[:x.find('.')])

In [ ]:
images.remove_columns(['path', 'filename'])

In [ ]:
images.column_types()

In [ ]:
images['id'] = images['id'].apply(lambda x: int(x))

### COMBINE IMAGES WITH TARGET DATA

In [ ]:
y_combined.reset_index(inplace=True)

In [ ]:
target = gl.SFrame(data=y_combined)

In [ ]:
images = images.join(target, on=['owner', 'id'], how='inner')

In [ ]:
images['user_is_pro'] = images['user_is_pro'].apply(lambda x: int(x))
images['user_can_buy_pro'] = images['user_can_buy_pro'].apply(lambda x: int(x))
images['user_total_views'] = images['user_total_views'].apply(lambda x: int(x))
images['image_views'] = images['image_views'].apply(lambda x: int(x))

### RESIZE IMAGES

In [4]:
images['resized_image'] = gl.image_analysis.resize(images['image'], 800, 600, channels=3)

extractor = gl.feature_engineering.DeepFeatureExtractor(features='resized_image', output_column_prefix='deep_')

extractor = extractor.fit(images)

extracted_model = extractor['model']

images = extractor.transform(images)

### Save Data, then Train-Test-Split

In [5]:
images.save('/home/ubuntu/data/GL_BUILDINGS_MODELING_DATA_RESIZED')

In [6]:
images.head(1)

Columns:
	image	Image
	owner	str
	id	int
	user_is_pro	float
	user_can_buy_pro	float
	user_total_views	float
	image_ncomments	int
	image_nfavs	float
	image_nsets	int
	image_npools	int
	image_views	float
	image_views_quantized	int
	user_total_views_quantized	int
	image_nfavs_quantized	int
	image_ncomments_quantized	int
	image_nsets_quantized	int
	image_npools_quantized	int
	resized_image	Image

Rows: 1

Data:
+--------------------------+-----------------+-------------+-------------+------------------+
|          image           |      owner      |      id     | user_is_pro | user_can_buy_pro |
+--------------------------+-----------------+-------------+-------------+------------------+
| Height: 1066 Width: 1600 | 49503002894@N01 | 20924375303 |     1.0     |       0.0        |
+--------------------------+-----------------+-------------+-------------+------------------+
+------------------+-----------------+-------------+-------------+--------------+
| user_total_views | image_ncomments | image_nfavs | image_nsets | image_npools |
+------------------+-----------------+-------------+-------------+--------------+
|     106299.0     |        0        |     0.0     |      1      |      0       |
+------------------+-----------------+-------------+-------------+--------------+
+-------------+-----------------------+----------------------------+-----------------------+
| image_views | image_views_quantized | user_total_views_quantized | image_nfavs_quantized |
+-------------+-----------------------+----------------------------+-----------------------+
|     41.0    |           2           |             5              |           1           |
+-------------+-----------------------+----------------------------+-----------------------+
+---------------------------+-----------------------+------------------------+
| image_ncomments_quantized | image_nsets_quantized | image_npools_quantized |
+---------------------------+-----------------------+------------------------+
|             1             |           1           |           1            |
+---------------------------+-----------------------+------------------------+
+------------------------+
|     resized_image      |
+------------------------+
| Height: 600 Width: 800 |
+------------------------+
[1 rows x 18 columns]

In [7]:
images_train, images_test = images.random_split(0.8)

### LOAD MODEL IF NEED BE

In [3]:
images = gl.load_sframe('/home/ubuntu/data/GL_BUILDINGS_MODELING_DATA_RESIZED')
#images_train, images_test = images.random_split(0.8)

### NEURAL NET MODEL

In [ ]:
images_test.head(1)

In [ ]:
network = gl.neuralnet_classifier.create(images_train, target='user_is_pro', features=['resized_image'],
                                         max_iterations=100, network=None, validation_set=images_test,
                                         class_weights='auto', metric='auto', random_crop=False,
                                         input_shape=None, random_mirror=False, learning_rate=0.001, momentum=0.9,
                                         l2_regularization=0.0005, bias_l2_regularization=0.0, init_random='gaussian',
                                         init_sigma=0.01, init_bias=0.0,
                                         model_checkpoint_path='/home/ubuntu/data/GL_BUILDINGS_MODEL_CHECKPOINT',
                                         model_checkpoint_interval=5)

In [ ]:
network.layers

In [ ]:
cust_network_layers = list()
#cust_network_layers.append(gl.deeplearning.layers.FlattenLayer())
cust_network_layers.append(gl.deeplearning.layers.FullConnectionLayer(100, init_sigma=0.5))
cust_network_layers.append(gl.deeplearning.layers.RectifiedLinearLayer())
cust_network_layers.append(gl.deeplearning.layers.DropoutLayer(0.5))
cust_network_layers.append(gl.deeplearning.layers.FullConnectionLayer(200, init_sigma=.1))
cust_network_layers.append(gl.deeplearning.layers.TanhLayer())
cust_network_layers.append(gl.deeplearning.layers.FullConnectionLayer(150, init_sigma=.1))
cust_network_layers.append(gl.deeplearning.layers.TanhLayer())
cust_network_layers.append(gl.deeplearning.layers.FullConnectionLayer(2, init_sigma=0.5))
cust_network_layers.append(gl.deeplearning.layers.SoftmaxLayer())

custom_network = gl.deeplearning.NeuralNet()
#custom_network.params['max_iterations'] = 20
custom_network.layers = cust_network_layers
custom_network.verify(input_shape=[1, 1, 4096], output_shape=2)

In [ ]:
model_neuralnetclassifier_ispro = gl.neuralnet_classifier.create(images_train, target='user_is_pro',
                                                                 features=['deep_.resized_image'],
                                                                 network=custom_network, max_iterations=100,
                                                                 validation_set=images_test, batch_size=1000,
                                                                 model_checkpoint_path='neural_network_model',
                                                                 model_checkpoint_interval=5
                                                                 )

In [ ]:
model_neuralnetclassifier_ispro.extract_features() # WHAT IS THIS?!

In [ ]:
neuralnetclassifier_results = model_neuralnetclassifier_ispro.evaluate(images_test)

In [ ]:
neuralnetclassifier_results.keys()

In [ ]:
neuralnetclassifier_results['accuracy']

In [ ]:
neuralnetclassifier_results['confusion_matrix']

In [ ]:
nn_test_predictions = model_neuralnetclassifier_ispro.predict(images_test)

In [ ]:
type(nn_test_predictions)

In [ ]:
gl.deeplearning.get_builtin_neuralnet('imagenet')